# Attribute Overlay Tutorial

This notebook gives an overview of what the attribute overlay is, why it exists and how to use it efficiently.

The attribute overlay has three main design goals
 * Allow overlay users to find out what is inside an overlay in a consistent manner
 * Provide a simple way for developers of new hardware designs to test new IP
 * Facilitate reuse of IP between Overlays
 
This tutorial is primarily designed to demonstrate the final two points, walking through the process of interacting with a new IP, developing a driver, and finally building a more complex system from multiple IP blocks

## Developing a Single IP

For this first example we are going to use a simple design with a single IP contained in it. This IP was developed using HLS and adds two 32-bit integers together. The full code for the accelerator is:

```C++
void add(int a, int b, int& c) {
#pragma HLS INTERFACE ap_ctrl_none port=return
#pragma HLS INTERFACE s_axilite port=a
#pragma HLS INTERFACE s_axilite port=b
#pragma HLS INTERFACE s_axilite port=c

	c = a + b;
}
```

With a block diagram consisting solely of the HLS IP and required glue logic to connect it to the ZYNQ7 IP

![Simple Block Diagram](../images/attribute1.png)

To interact with the IP first we need to load the overlay containing the IP.

In [1]:
from pynq import Overlay

overlay = Overlay('/home/xilinx/attribute_test1.bit')

Creating the overlay will automatically download it. We can now use `help` to find out what is in the overlay.

In [2]:
help(overlay)

Help on DocumentedOverlay in module pynq.overlay object:

class DocumentedOverlay(DefaultOverlay)
 |  Default documentation for overlay /home/xilinx/attribute_test1.bit. The following
 |  attributes are available on this overlay:
 |  
 |  IP Blocks
 |  ----------
 |  scalar_add           : pynq.overlay.DefaultIP
 |  
 |  Hierarchies
 |  -----------
 |  None
 |  
 |  Interrupts
 |  ----------
 |  None
 |  
 |  GPIO Outputs
 |  ------------
 |  None
 |  
 |  Method resolution order:
 |      DocumentedOverlay
 |      DefaultOverlay
 |      pynq.pl.PL
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from DefaultOverlay:
 |  
 |  __dir__(self)
 |      __dir__() -> list
 |      default dir() implementation
 |  
 |  __getattr__(self, key)
 |      Overload of __getattr__ to return a driver for an IP or
 |      hierarchy. Throws an `RuntimeError` if the overlay is 

All of the entries are accessible via attributes on the overlay class with the specified driver. Accessing the `scalar_add` attribute of the will create a driver for the IP - as there is no driver currently known for the `Add` IP core `DefaultIP` driver will be used so we can interact with IP core.

In [3]:
add_ip = overlay.scalar_add
help(add_ip)

Help on DefaultIP in module pynq.overlay object:

class DefaultIP(builtins.object)
 |  Driver for an IP without a more specific driver
 |  
 |  This driver wraps an MMIO device and provides a base class
 |  for more specific drivers written later. It also provides
 |  access to GPIO outputs and interrupts inputs via attributes. More specific
 |  drivers should inherit from `DefaultIP` and include a
 |  `bindto` entry containing all of the IP that the driver
 |  should bind to. Subclasses meeting these requirements will
 |  automatically be registered.
 |  
 |  Attributes
 |  ----------
 |  mmio : pynq.MMIO
 |      Underlying MMIO driver for the device
 |  _interrupts : dict
 |      Subset of the PL.interrupt_pins related to this IP
 |  _gpio : dict
 |      Subset of the PL.gpio_dict related to this IP
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  read(self, offset=0)
 |      Read from 

Reading the documentation generated by HLS tells us that use the core we need to write the two arguments to offset `0x10` and `0x18` and read the result back from `0x20`.

In [4]:
add_ip.write(0x10, 4)
add_ip.write(0x18, 5)
add_ip.read(0x20)

9

## Creating a Driver

While the `UnknownIP` driver is useful for determining that the IP is working it is not the most user-friendly API to expose to the eventual end-users of the overlay. Ideally we want to create an IP-specific driver exposing a single `add` function to call the accelerator. The only requirements for a driver is that it can be constructed with a single argument called `description` which is the entry in the `PL.ip_dict` for the IP. It is also recommend that the custom driver inherit from `UnknownIP` so that the experience for the user is consistent. Inheriting also allows us to use the `read` and `write` functions as in our standalone code.

In [5]:
from pynq import DefaultIP

class AddDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
    
    bindto = ['xilinx.com:hls:add:1.0']
    
    def add(self, a, b):
        self.write(0x10, a)
        self.write(0x18, b)
        return self.read(0x20)
    
add_ip = AddDriver(overlay.ip_dict['scalar_add'])
add_ip.add(10,20)

30

While this driver works and can be instantiated manually it would be preferable for it to be created automatically when a user interacts with the IP. For that there is a `register_type` function which allows to associate a driver with an IP class. The IP class for the scalar add IP is `xilinx.com:hls:add:1.0` so lets register the driver.

Now if we reload the overlay and query the help again we can see that our new driver is bound to the IP.

In [6]:
overlay = Overlay('/home/xilinx/attribute_test1.bit')
help(overlay)

Help on DocumentedOverlay in module pynq.overlay object:

class DocumentedOverlay(DefaultOverlay)
 |  Default documentation for overlay /home/xilinx/attribute_test1.bit. The following
 |  attributes are available on this overlay:
 |  
 |  IP Blocks
 |  ----------
 |  scalar_add           : __main__.AddDriver
 |  
 |  Hierarchies
 |  -----------
 |  None
 |  
 |  Interrupts
 |  ----------
 |  None
 |  
 |  GPIO Outputs
 |  ------------
 |  None
 |  
 |  Method resolution order:
 |      DocumentedOverlay
 |      DefaultOverlay
 |      pynq.pl.PL
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from DefaultOverlay:
 |  
 |  __dir__(self)
 |      __dir__() -> list
 |      default dir() implementation
 |  
 |  __getattr__(self, key)
 |      Overload of __getattr__ to return a driver for an IP or
 |      hierarchy. Throws an `RuntimeError` if the overlay is not 

And we can access the same way as before

In [7]:
overlay.scalar_add.add(15,20)

35

## Reusing IP

Suppose we or someone else develops a new overlay and wants to reuse the existing IP. As long as they import the python file contain the `register_type` command all of the drivers will be automatically created. As an example consider the next design which, among other things includes a renamed version of the `scalar_add` IP.

![Second Block Diagram](../images/attribute2.png)

Running `help` on the new overlay shows that the driver is still bound.

In [8]:
overlay = Overlay('/home/xilinx/attribute_test2.bit')
help(overlay)

Help on DocumentedOverlay in module pynq.overlay object:

class DocumentedOverlay(DefaultOverlay)
 |  Default documentation for overlay /home/xilinx/attribute_test2.bit. The following
 |  attributes are available on this overlay:
 |  
 |  IP Blocks
 |  ----------
 |  old_add              : __main__.AddDriver
 |  
 |  Hierarchies
 |  -----------
 |  const_multiply       : pynq.overlay.DefaultHierarchy
 |  
 |  Interrupts
 |  ----------
 |  None
 |  
 |  GPIO Outputs
 |  ------------
 |  None
 |  
 |  Method resolution order:
 |      DocumentedOverlay
 |      DefaultOverlay
 |      pynq.pl.PL
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from DefaultOverlay:
 |  
 |  __dir__(self)
 |      __dir__() -> list
 |      default dir() implementation
 |  
 |  __getattr__(self, key)
 |      Overload of __getattr__ to return a driver for an IP or
 |      hierarchy.

## IP Hierarchies

The block diagram above also contains a hierarchy looking like this:

![Hierarchy](../images/hierarchy.png)

Containing a custom IP for multiple a stream of numbers by a constant and a DMA engine for transferring the data. As streams are involved and we need correctly handle `TLAST` for the DMA engine the HLS code is a little more complex with additional pragmas and types but the complete code is still relatively short.

```C
typedef ap_axiu<32,1,1,1> stream_type;

void mult_constant(stream_type* in_data, stream_type* out_data, ap_int<32> constant) {
#pragma HLS INTERFACE s_axilite register port=constant
#pragma HLS INTERFACE ap_ctrl_none port=return
#pragma HLS INTERFACE axis port=in_data
#pragma HLS INTERFACE axis port=out_data
	out_data->data = in_data->data * constant;
	out_data->dest = in_data->dest;
	out_data->id = in_data->id;
	out_data->keep = in_data->keep;
	out_data->last = in_data->last;
	out_data->strb = in_data->strb;
	out_data->user = in_data->user;

}
```

Looking at the HLS generated documentation we again discover that to set the constant we need to set the register at offset `0x10` so we can write a simple driver for this purpose

In [9]:
class ConstantMultiplyDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
    
    bindto = ['Xilinx:hls:mult_constant:1.0']
    
    @property
    def constant(self):
        return self.read(0x10)
    
    @constant.setter
    def constant(self, value):
        self.write(0x10, value)

The DMA engine driver is already included inside the PYNQ driver so nothing special is needed for that other than ensuring the module is imported. Reloading the overlay will make sure that our newly written driver is available for use.

In [10]:
import pynq.lib.dma

overlay = Overlay('/home/xilinx/attribute_test2.bit')

dma = overlay.const_multiply.multiply_dma
multiply = overlay.const_multiply.multiply

The DMA driver transfers numpy arrays allocated using the `xlnk` driver. Lets test the system by multiplying 5 numbers by 3.

In [11]:
from pynq import Xlnk
import numpy as np

xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(5,), dtype=np.uint32)
out_buffer = xlnk.cma_array(shape=(5,), dtype=np.uint32)

for i in range(5):
    in_buffer[i] = i

multiply.constant = 3
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()

out_buffer

CMABuffer([ 0,  3,  6,  9, 12], dtype=uint32)

While this is one way to use the IP, it still isn't exactly user-friendly. It would be preferable to treat the entire hierarchy as a single entity and write a driver that hides the implementation details. The attribute overlay allows for drivers to be written against hierarchies as well as IP but the details are slightly different.

The driver writer should provide a function which takes the hierarchy name and a subset of the `PL.ip_dict` containing the hierarchies IP and return either a class instance if the driver can be instantiated or `None` if not. It is recommended that hierarchy drivers inherit from the `Hierarchy` class as this provides easy access to the contained IP.

For our constant multiply hierarchy this would look something like:

In [15]:
from pynq import DefaultHierarchy

class StreamMultiplyDriver(DefaultHierarchy):
    def __init__(self, description):
        super().__init__(description)
        
    def stream_multiply(self, stream, constant):
        self.multiply.constant = constant
        with xlnk.cma_array(shape=(len(stream),), dtype=np.uint32) as in_buffer,\
             xlnk.cma_array(shape=(len(stream),), dtype=np.uint32) as out_buffer:
            for i, v, in enumerate(stream):
                in_buffer[i] = v
            self.multiply_dma.sendchannel.transfer(in_buffer)
            self.multiply_dma.recvchannel.transfer(out_buffer)
            self.multiply_dma.sendchannel.wait()
            self.multiply_dma.recvchannel.wait()
            result = out_buffer.copy()
        return result

    @staticmethod
    def checkhierarchy(description):
        if 'multiply_dma' in description['ip'] and 'multiply' in description['ip']:
            return True
        return False

We can now reload the overlay and ensure the higher-level driver is loaded

In [16]:
overlay = Overlay('/home/xilinx/attribute_test2.bit')
help(overlay)

Help on DocumentedOverlay in module pynq.overlay object:

class DocumentedOverlay(DefaultOverlay)
 |  Default documentation for overlay /home/xilinx/attribute_test2.bit. The following
 |  attributes are available on this overlay:
 |  
 |  IP Blocks
 |  ----------
 |  old_add              : __main__.AddDriver
 |  
 |  Hierarchies
 |  -----------
 |  const_multiply       : __main__.StreamMultiplyDriver
 |  
 |  Interrupts
 |  ----------
 |  None
 |  
 |  GPIO Outputs
 |  ------------
 |  None
 |  
 |  Method resolution order:
 |      DocumentedOverlay
 |      DefaultOverlay
 |      pynq.pl.PL
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from DefaultOverlay:
 |  
 |  __dir__(self)
 |      __dir__() -> list
 |      default dir() implementation
 |  
 |  __getattr__(self, key)
 |      Overload of __getattr__ to return a driver for an IP or
 |      hierarchy.

and use it

In [17]:
overlay.const_multiply.stream_multiply([1,2,3,4,5], 5)

CMABuffer([ 5, 10, 15, 20, 25], dtype=uint32)

## Overlay Customisation

While the default attribute overlay is sufficient for many use cases, some overlays will require more customisation to provide a user-friendly API. As an example the default AXI GPIO drivers expose channels 1 and 2 as separate attributes meaning that accessing the LEDs in the base overlay requires the following contortion

In [18]:
base = Overlay('base.bit')
base.swsleds_gpio.channel2[0].on()

SyntaxError: invalid syntax (audio.py, line 68)

To mitigate this the overlay developer can provide a custom class for their overlay to expose the subsystems in a more user-friendly way.  The base overlay includes custom overlay class which performs the following functions:
 * Make the AXI GPIO devices better named and range/direction restricted
 * Make the IOPs accessible through the `pmoda`, `pmodb` and `ardiuno` names
 * Create a speical class to interact with RGB LEDs
 
The result is that the LEDs can be accessed like:

In [17]:
base = Overlay('base.bit')
base.leds[0].on()

Using a well defined class also allows for custom docstrings to be provided also helping end users.

In [18]:
help(base)

Help on BaseOverlay object:

class BaseOverlay(pynq.overlay.DefaultOverlay)
 |  The Base overlay for the Pynq-Z1
 |  
 |  This overlay is designed to interact with all of the on board peripherals
 |  and external interfaces of the Pynq-Z1 board. It exposes the following
 |  attributes:
 |  
 |  Attributes
 |  ----------
 |  pmoda : IOP
 |       IO processor connected to the PMODA interface
 |  pmodb : IOP
 |       IO processor connected to the PMODB interface
 |  arduino : IOP
 |       IO processor connected to the Arduino/ChipKit interface
 |  leds : AxiGPIO
 |       4-bit output GPIO for interacting with the green LEDs LD0-3
 |  buttons : AxiGPIO
 |       4-bit input GPIO for interacting with the buttons BTN0-3
 |  switches : AxiGPIO
 |       2-bit input GPIO for interacting with the switches SW0 and SW1
 |  rgbleds : [pynq.board.RGBLED]
 |       Wrapper for GPIO for LD4 and LD5 multicolour LEDs
 |  video : pynq.lib.video.HDMIWrapper
 |       HDMI input and output interfaces
 |  audi

### Creating a custom overlay

Custom overlay classes should inherit from `pynq.UnknownOverlay` taking a single construction parameter which is the full path of the bitstream file. This path should be passed to `super().__init__()` at the start of `__init__` to initialise the attributes of the Overlay. This example is designed to go with our attribute_test2 overlay and adds a function to more easily call the multiplication function

In [17]:
from pynq import DefaultOverlay
class TestOverlay(DefaultOverlay):
    def __init__(self, bitfile):
        super().__init__(bitfile)
    
    def multiply(self, stream, constant):
        return self.const_multiply.stream_multiply(stream, constant)

To test our new overlay class we can pass it in as a parameter to `pynq.Overlay`

In [18]:
overlay = Overlay('/home/xilinx/attribute_test2.bit', TestOverlay)
overlay.multiply([2,3,4,5,6], 4)

CMABuffer([ 8, 12, 16, 20, 24], dtype=uint32)

To have this happen automatically we need to place our new class inside a .py file alongside the bitstream. It is also worth importing all of the drivers needed to ensure that they are loaded before the overlay is constructed. Finally we need to create a variable `Overlay` pointing to the class that should be constructed.

The final version of `attribute_test2.py` would look like:

```python
import pynq.lib.dma
import ouriplib
from pynq import DefaultOverlay

class TestOverlay(DefaultOverlay):
    def __init__(self, bitfile):
        super().__init__(bitfile)
    
    def multiply(self, stream, constant):
        return self.const_multiply.stream_multiply(stream, constant)

Overlay = TestOverlay
```

## Included Drivers

The pynq library includes a number of drivers as part of the `pynq.lib` package. These include
 * AXI GPIO
 * AXI DMA (simple mode only)
 * AXI VDMA
 * AXI Interrupt Controller (internal use)
 * Pynq-Z1 Audio IP
 * Pynq-Z1 HDMI IP
   * Color convert IP
   * Pixel format conversion
   * HDMI input and output frontends
 * Pynq Microblaze program loading